In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from mtflearn.clustering import gmm_lbs
from mtflearn.io import load_image
from stemplot import imshow, colors_from_lbs
from mtflearn.features import ZPs, get_characteristic_length, local_max, KeyPoints
from mtflearn.utils import normalize_image, remove_bg
from mtflearn.manifold import ForceGraph8
from sklearn.manifold import TSNE as tsne
from stemplot.colors import color_palette,xy2colors
from skimage.transform import resize
from stemplot.interactive import interactive_clusters
import umap

In [2]:
img = load_image('ADF2_27.dm4')
original_pts = np.load('Co 3 pts.npy')
lbs = np.load('Co 3 lbs.npy').astype(int)

In [3]:
if img.shape[0] == 2048:
    img = resize(img, (1024, 1024), order=0)
img = normalize_image(img)
# mask1 = np.logical_or(lbs==0, lbs==1)
# mask1 = lbs==0
mask2 = np.logical_or(lbs==2, lbs==3)
mask1 = np.logical_or(lbs==0, lbs==1)
size = 29

In [4]:
kp = KeyPoints(original_pts, img, size)
original_ps = kp.extract_patches(size)
zps = ZPs(n_max=12, size=original_ps.shape[1])
moments = zps.fit_transform(original_ps)

In [5]:
pts = original_pts[mask1]
ps = original_ps[mask1]

In [6]:
Se2_lbs = lbs[mask1]

### TSNE

In [7]:
tsne_X1 = X1 = (moments.data)[mask1][:, 1:]

In [8]:
tsne_res = tsne(n_components=2).fit_transform(tsne_X1)

In [7]:
tsne_lbs = gmm_lbs(tsne_res,10)

In [8]:
app = interactive_clusters(tsne_res, img, pts[mask1], ps[mask1], lbs=tsne_lbs, s=1)
plt.savefig('tsne.png',transparent=True)

In [18]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = tsne_lbs,cmap = 'tab10',s=7)
ax.axis('off')
plt.savefig('img with tsne color.png',dpi = 700,transparent=True)

In [18]:
fig,ax = plt.subplots(2,5, figsize=(15,5.5))
for i in range(10):
    select_ps = np.mean(ps[mask1][tsne_lbs == i],axis = 0)
    ax[i//5,i%5].imshow(select_ps,cmap= color_palette('C{}'.format(i)))
    ax[i//5,i%5].axis('off')
plt.savefig('tsne ps.png',dpi = 700,transparent=True)

In [19]:
imshow(select_ps,cmap= color_palette('C0'))

### FR

In [19]:
X = moments.select([0]).data
FR_X1 = X[mask1][:, 1:]

In [20]:
fg = ForceGraph8(X = FR_X1,
                 n_neighbors=5,
                 init_mode = 'pca',
                 num_iterations = 100,
                 num_negative_samples=5,
                 force_params1=(0, 2, 1, 1),
                 force_params2=(2, 5, 5, 1),
                 divide=0.5)

from time import time
t0 = time()
FR_res = fg.fit_transform(FR_X1)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [21]:
FR_lbs = gmm_lbs(FR_res,10)

In [44]:
app = interactive_clusters(FR_res, img, pts[mask1], ps[mask1], lbs=FR_lbs, s=1)
plt.savefig('fr.png',transparent=True)

In [45]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = FR_lbs,cmap = 'tab10',s=7)
ax.axis('off')
plt.savefig('img with FR color.png',dpi = 700,transparent=True)

In [22]:
fig,ax = plt.subplots(2,5, figsize=(15,5.5))
for i in range(10):
    select_ps = np.mean(ps[mask1][FR_lbs == i],axis = 0)
    ax[i//5,i%5].imshow(select_ps,cmap= color_palette('C{}'.format(i)))
    ax[i//5,i%5].axis('off')
plt.savefig('FR ps.png',dpi = 700,transparent=True)

### umap

In [6]:
umap_X1 = X1 = (moments.data)[mask1][:, 1:]
umap_res = umap.UMAP(n_components=2).fit_transform(umap_X1)

In [7]:
umap_lbs = gmm_lbs(umap_res,10)

In [8]:
app = interactive_clusters(umap_res, img, pts[mask1], ps[mask1], lbs=umap_lbs, s=1)
plt.savefig('umap.png',transparent=True)

In [9]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = umap_lbs,cmap = 'tab10',s=7)
ax.axis('off')
plt.savefig('img with umap color.png',dpi = 700,transparent=True)

In [13]:
fig,ax = plt.subplots(2,5, figsize=(15,5.5))
for i in range(10):
    select_ps = np.mean(ps[mask1][umap_lbs == i],axis = 0)
    ax[i//5,i%5].imshow(select_ps,cmap= color_palette('C{}'.format(i)))
    ax[i//5,i%5].axis('off')
plt.savefig('umap ps.png',dpi = 700,transparent=True)

### 

In [29]:
def xy2colors_fix_raduis(xy, v=0.9, return_rgb=True):
    # Calculate the angles from xy coordinates
    angles = (np.arctan2(xy[:, 1], xy[:, 0]) + np.pi) / (2 * np.pi)

    # Since we want colors to vary only with angle, we create a constant saturation
    # and use the same value for brightness.
    # The saturation here is set to 1 for maximum color intensity.
    saturation = np.ones_like(angles)  # S = 1 for all points
    value = np.ones_like(angles) * v   # V = v for all points

    # Construct the HSV color representation
    hsv_colors = np.vstack([angles, saturation, value]).T

    # Convert to RGB if required
    if return_rgb:
        return hsv_to_rgb(hsv_colors)
    else:
        return hsv_colors

def xy2colors_fix_angle(xy, v=0.9, return_rgb=True):
    # Compute the radius and normalize it
    radius = np.hypot(xy[:, 0], xy[:, 1])
    max_radius = np.max(radius)
    if max_radius > 0:
        radius = radius / max_radius

    # Since we want colors to vary only with radius, we create a constant hue
    # and use the same value for brightness.
    # The hue here is set based on the normalized radius.
    hue = radius  # H = radius for all points
    saturation = np.ones_like(radius)  # S = 1 for all points
    value = np.ones_like(radius) * v   # V = v for all points

    # Construct the HSV color representation
    hsv_colors = np.vstack([hue, saturation, value]).T

    # Convert to RGB if required
    if return_rgb:
        return hsv_to_rgb(hsv_colors)
    else:
        return hsv_colors

In [10]:
polar = xy2colors(tsne_res, v=0.9, return_rgb=True)

In [20]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.scatter(tsne_res[:,0],tsne_res[:,1],c=polar,s=5)
plt.savefig('tsne polar.png',transparent=True)

In [19]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = polar,s=7)
ax.axis('off')
plt.savefig('img with tsne polar color.png',dpi = 700,transparent=True)

In [24]:
r_polar = xy2colors_fix_raduis(tsne_res,v = 0.9,return_rgb=True)

In [26]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.scatter(tsne_res[:,0],tsne_res[:,1],c=r_polar,s=5)
plt.savefig('tsne polar fix raduis.png',transparent=True)

In [27]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = r_polar,s=7)
ax.axis('off')
plt.savefig('img with tsne polar color fix radius.png',dpi = 700,transparent=True)

In [30]:
a_polar = xy2colors_fix_angle(tsne_res,v = 0.9,return_rgb=True)

In [32]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.scatter(tsne_res[:,0],tsne_res[:,1],c=a_polar,s=5)
plt.savefig('tsne polar fix angle.png',transparent=True)

In [33]:
fig,ax = plt.subplots(figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(pts[mask1][:,0],pts[mask1][:,1],c = a_polar,s=7)
ax.axis('off')
plt.savefig('img with tsne polar color fix angle.png',dpi = 700,transparent=True)

In [7]:
def sector_indices(xy,sector_number = 11):
    angles = np.arctan2(xy[:, 1], xy[:, 0])
    sector_boundaries = np.linspace(-np.pi, np.pi, sector_number)
    grouped_indices = []
    for i in range(len(sector_boundaries)-1):
        start = sector_boundaries[i]
        end = sector_boundaries[i+1]
        indices = np.where((angles >= start) & (angles < end))
        grouped_indices.append(indices)
        # grouped_points.append(xy[indices])
    return grouped_indices

def radial_indices(xy,radial_number = 6):
    radius = np.hypot(xy[:, 0], xy[:, 1])
    max_radius = np.max(radius)
    radial_boundaries = np.linspace(0, max_radius, radial_number)
    grouped_indices = []
    for i in range(len(radial_boundaries)-1):
        start = radial_boundaries[i]
        end = radial_boundaries[i+1]
        indices = np.where((radius >= start) & (radius < end))
        grouped_indices.append(indices)
    return grouped_indices

def sector_features(xy,grouped_indices):
    grouped_features = []
    for indices in grouped_indices:
        grouped_features.append(xy[indices])
    return grouped_features

def sector_pts(pts,grouped_indices):
    grouped_pts = []
    for indices in grouped_indices:
        grouped_pts.append(pts[indices])
    return grouped_pts

def sector_ps(ps,grouped_indices,mean = True):
    grouped_ps = []
    for indices in grouped_indices:
        grouped_ps.append(ps[indices])
    new_ps = []
    if mean:
        for g in grouped_ps:
            new_ps.append(np.mean(g,axis = 0))
        return new_ps
    else:
        return grouped_ps


In [88]:
grouped_indices = sector_indices(tsne_res)
grouped_features = sector_pts(tsne_res,grouped_indices)
grouped_pts = sector_pts(pts,grouped_indices)
grouped_ps = sector_ps(ps,grouped_indices)

In [80]:
tsne_res.shape,pts.shape,ps.shape

((4114, 2), (4114, 2), (4114, 29, 29))

In [12]:
colors = plt.get_cmap('tab10')

In [94]:
for idx,[select_pts,select_features,select_ps] in enumerate(zip(grouped_pts,grouped_features,grouped_ps)):
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    ax[0].imshow(img,cmap = 'gray')
    ax[0].axis('off')
    ax[0].scatter(select_pts[:,0],select_pts[:,1],color = colors(idx),s = 7)
    ax[1].scatter(tsne_res[:,0],tsne_res[:,1],color='black',s = 3)
    ax[1].scatter(select_features[:,0],select_features[:,1],color = colors(idx),s=3)
    ax[2].imshow(select_ps,cmap = 'gray',vmin = select_ps.min(),vmax = select_ps.max()+0.1)
    ax[2].axis('off')
    plt.savefig('sector {}.png'.format(idx),dpi = 700)
    plt.close(fig)

In [18]:
grouped_indices = radial_indices(tsne_res,radial_number = 8)
grouped_features = sector_pts(tsne_res,grouped_indices)
grouped_pts = sector_pts(pts,grouped_indices)
grouped_ps = sector_ps(ps,grouped_indices)

In [16]:
for idx,[select_pts,select_features,select_ps] in enumerate(zip(grouped_pts,grouped_features,grouped_ps)):
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    ax[0].imshow(img,cmap = 'gray')
    ax[0].axis('off')
    ax[0].scatter(select_pts[:,0],select_pts[:,1],color = colors(idx),s = 7)
    ax[1].scatter(tsne_res[:,0],tsne_res[:,1],color='black',s = 3)
    ax[1].scatter(select_features[:,0],select_features[:,1],color = colors(idx),s=3)
    ax[2].imshow(select_ps,cmap = 'gray',vmin = select_ps.min(),vmax = select_ps.max()+0.1)
    ax[2].axis('off')
    plt.savefig('radial {}.png'.format(idx),dpi = 700)
    plt.close(fig)

In [17]:
len(grouped_ps)

7

In [9]:
from sklearn.neighbors import NearestNeighbors

In [10]:
Se2_lbs.shape,tsne_res.shape

((4200,), (4200, 2))

In [11]:
pts1 = pts[Se2_lbs == 0]#Se2 positions
pts2 = pts[Se2_lbs == 1]#Vacancy positions

X11 = tsne_X1[Se2_lbs == 0]
X12 = tsne_X1[Se2_lbs == 1]

tsne_res = tsne(n_components=2).fit_transform(X11)

In [12]:
tsne_res.shape,pts1.shape,pts2.shape

((4114, 2), (4114, 2), (86, 2))

In [13]:
nn = NearestNeighbors(n_neighbors=2)
nn.fit(pts1)

distances, indices = nn.kneighbors(pts1)
nearest_distances = distances[:, 1]
average_distance = np.mean(nearest_distances)
radius = int(average_distance*1.3)
print("Radius of further points nearest neighbor:", radius)

Radius of further points nearest neighbor: 19


In [14]:
def sort_pts(pts,p):
    vectors = pts - p
    angles = np.arctan2(vectors[:, 1], vectors[:, 0])
    sorted_indices = np.argsort(angles)
    return sorted_indices

In [15]:
select_pts1 = []
select_theta1 = []
angles = np.arctan2(tsne_res[:, 1], tsne_res[:, 0])
nn = NearestNeighbors(radius=radius)
nn.fit(pts1)
neighbor_limit = 7
for i,p in enumerate(pts1):
    distances, indices = nn.radius_neighbors([p])

    if i in indices[0]:
        indices = indices[0][indices[0] != i]
        distances = distances[0][distances[0] != i]

    if len(indices) >= neighbor_limit - 1:
        select_pts = pts1[indices]
        select_angles = angles[indices]
        sort_idx = sort_pts(select_pts,p)
        sort_angles = select_angles[sort_idx]
        diffs = np.diff(sort_angles)
        result = np.append(diffs,  sort_angles[-1] - sort_angles[0])
        select_pts1.append(p)
        select_theta1.append(result)
select_pts1 = np.array(select_pts1)
select_theta1 = np.array(select_theta1)

In [16]:
select_pts2 = []
select_theta2 = []
neighbor_limit = 6
for i,p in enumerate(pts2):
    distances, indices = nn.radius_neighbors([p])
    
    indices = indices[0]
    distances = distances[0]

    if len(indices) >= neighbor_limit:
        select_pts = pts1[indices]
        select_angles = angles[indices]
        sort_idx = sort_pts(select_pts,p)
        sort_angles = select_angles[sort_idx]
        diffs = np.diff(sort_angles)
        result = np.append(diffs,  sort_angles[-1] - sort_angles[0])
        select_pts2.append(p)
        select_theta2.append(result)
select_pts2 = np.array(select_pts2)
select_theta2 = np.array(select_theta2)

In [17]:
mean_theta1 = np.mean(select_theta1,axis = 1)
mean_theta2 = np.mean(select_theta2,axis = 1)

In [18]:
mean_theta = np.hstack([mean_theta1,mean_theta2])
all_pts = np.vstack([select_pts1,select_pts2])

In [19]:
norm_theta = mean_theta-mean_theta.min()
norm_theta = norm_theta/norm_theta.max()

In [36]:
new_cmap = plt.get_cmap('bwr')(norm_theta)

In [23]:
plt.style.use('default')

In [37]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
cmap = plt.get_cmap('bwr')
# norm = Normalize(vmin=norm_theta.min(), vmax=norm_theta.max())
norm = Normalize(vmin=-np.pi, vmax=np.pi)
fig,ax = plt.subplots(1,1,figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(all_pts[:,0],all_pts[:,1],c = new_cmap,s = 7)
ax.axis('off')
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# 使用 make_axes_locatable 创建一个与当前轴相关的分区
divider = make_axes_locatable(ax)
# 在右侧添加一个新的轴，用于放置 colorbar
cax = divider.append_axes("right", size="5%", pad=0.05)

# 添加颜色条到新的轴上，并且设置其高度与原图相同
cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
ticks = [-np.pi,-2,-1, 0,1,2, np.pi]  # 定义ticks的位置
tick_labels = [r'$-\pi$',r'$-2$' ,r'$-1$' ,r'$0$',r'$1$' ,r'$2$' , r'$\pi$']  # 定义ticks的标签，这里使用 LaTeX 格式

cbar.set_ticks(ticks)
cbar.set_ticklabels(tick_labels)

# 如果你想要更加精确地控制colorbar的大小，可以调整size参数
# cax = divider.append_axes("right", size="100%", pad=0.05) # 这会使得colorbar宽度等于主图宽度
plt.savefig('delta theta bwr cmap with colorbar.png',dpi = 700,transparent = True)

In [22]:
fig,ax = plt.subplots(1,1)
ax.hist(norm_theta,bins=100,color = 'blue')

(array([ 3.,  4., 10.,  8.,  8., 12.,  9.,  7., 10., 13., 13., 13., 21.,
        15., 20., 14., 24., 18., 16., 18., 22., 20., 21., 34., 30., 32.,
        28., 30., 32., 26., 51., 33., 34., 41., 56., 43., 43., 50., 59.,
        57., 43., 66., 63., 57., 63., 77., 64., 77., 90., 88., 91., 93.,
        67., 85., 65., 73., 62., 67., 60., 63., 72., 55., 58., 57., 57.,
        61., 52., 40., 37., 26., 48., 42., 45., 27., 26., 16., 22., 13.,
        20., 26., 16., 23., 24., 20., 16., 11., 17., 20., 10., 12., 11.,
        12.,  8., 11., 11.,  7., 12.,  9.,  4.,  3.]),
 array([0.        , 0.01      , 0.02      , 0.03      , 0.04      ,
        0.05      , 0.06      , 0.07      , 0.08      , 0.09      ,
        0.1       , 0.11      , 0.12      , 0.13      , 0.14      ,
        0.15000001, 0.16      , 0.17      , 0.18000001, 0.19      ,
        0.2       , 0.20999999, 0.22      , 0.23      , 0.23999999,
        0.25      , 0.25999999, 0.27000001, 0.28      , 0.28999999,
        0.30000001, 0.31  

In [100]:
fig,ax = plt.subplots(1,1)
ax.hist(np.mean(select_theta2,axis = 1),bins = 20)

(array([ 2.,  3.,  1.,  3.,  1.,  4.,  4.,  5.,  5., 12., 12.,  4.,  7.,
         5.,  2.,  0.,  1.,  2.,  1.,  2.]),
 array([-1.85589659, -1.66556358, -1.47523069, -1.28489769, -1.09456468,
        -0.90423179, -0.71389878, -0.52356583, -0.33323285, -0.14289989,
         0.04743308,  0.23776604,  0.42809901,  0.61843199,  0.80876493,
         0.99909794,  1.18943083,  1.37976384,  1.57009685,  1.76042974,
         1.95076275]),
 <BarContainer object of 20 artists>)

In [91]:
fig, ax = plt.subplots(1,1)
ax.imshow(img,cmap = 'gray')
for p1 in pts1[indices]:
    ax.scatter(p1[0],p1[1],color = 'red',s = 7)
ax.scatter(p[0],p[1],color = 'b',s = 7)

In [76]:
pts1.shape

(4114, 2)